# Was ist PyTables?

PyTables is a package for managing hierarchical datasets and designed to efficiently and easily cope with extremely large amounts of data. <br> <a href:'http://www.pytables.org/'> API </a>

# Was sind 'hierarchical datasets'?

PyTables arbeitet mit Daten im HDF5 Format. HDF5 ist ein Format von der HDF Group entwickelt und von ihnen auch gepflegt wird. HDF5 nutzt immer einen Namen als Speicher. in diesem Speicher kann man Gruppen anlegen und in jeder Gruppe die dazugehörigen Daten Speichern.

<img src='HDF5.jpg'>

Die Daten im HDF Format können wie im Dask Beispiel mit Compression gespeichert werden.

# Unterschied zwischen Dask und PyTables

PyTables ist für etwas ganz anderes als Dask konzipiert. Während Dask auf Pandas beruht und hauptsächlich für CSV und Excel Tabellen gedacht ist, ist PyTables auf das Speichern großer Datenmengen ausgerichtet.<br>
Während Dask Spalten und Reihenweise Aufrufe ermöglich kann PyTables nur Spalte für Spalte iterieren.<br>
Auch ist PyTables nicht flexibel wir Dask wo neue Spalten hinzugefügt werden können.<br>


# Wofür brauche ich dann PyTables?

PyTables soll zum Speichern von großen Datenmengen genutzt werden. Im HDF5 Format kann man auch eigene Datenformate erstellen. Die NASA speichert große Mengen an Daten im <a href='https://earthdata.nasa.gov/user-resources/standards-and-references/hdf5'> HDF-Format</a>. Auf Youtube gibt es Beispiele zum Analysieren von Finanzdaten mit Hilfe von PyTables. <a href='http://hilpisch.com/Big_Financial_Data.html#/'> Hier als Beispiel</a>



In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/_JrZUm9ZHIw" frameborder="0" allowfullscreen></iframe>

HDF5 und PyTables haben eine sehr steile Lernkurve, benötigen aber vorher ein bisschen Recherche und Zeit. Da ich den zeitlichen Rahmen nicht sprengen will, bleib ich bei einer kurzen Einführung.

## Beispiel

Wir nehmen an wir haben einen Teilchenbeschleuniger und wollen die gemessenen Partikel speichern.
name = Name<br>
idnumber = id <br>
ADC+TDCcount = Zeiten in Bit Format<br>
grid_i+j= Positionen im Raum<br>
pressure + energy = Druck im Raum Energie + des Partikel<br>
Der Aufbau ähnelt dem einer Datenbank


In [30]:
from tables import *
class Particle(IsDescription):
    name      = StringCol(16)   # 16-character String
    idnumber  = Int64Col()      # Signed 64-bit integer
    ADCcount  = UInt16Col()     # Unsigned short integer
    TDCcount  = UInt8Col()      # unsigned byte
    grid_i    = Int32Col()      # 32-bit integer
    grid_j    = Int32Col()      # 32-bit integer
    pressure  = Float32Col()    # float  (single-precision)
    energy    = Float64Col()    # double (double-precision)

Erstellen einer Datei im HDF5 Format. Dateiname = tutorial1, Selbstbeschreibung = Test file

In [31]:
h5file = open_file("tutorial1.h5", mode = "w", title = "Test file")

Erstellen einer Gruppe in der Datei. Name = detector, Selbstbeschreibung = Detector Information

In [32]:
group = h5file.create_group("/", 'detector', 'Detector information')

Erstellen einer Tabelle in der Gruppe detector. Name = readout Selbstbeschreibung = Readout example

In [33]:
table = h5file.create_table(group, 'readout', Particle, "Readout example")

In [34]:
h5file

File(filename=tutorial1.h5, title='Test file', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) 'Test file'
/detector (Group) 'Detector information'
/detector/readout (Table(0,)) 'Readout example'
  description := {
  "ADCcount": UInt16Col(shape=(), dflt=0, pos=0),
  "TDCcount": UInt8Col(shape=(), dflt=0, pos=1),
  "energy": Float64Col(shape=(), dflt=0.0, pos=2),
  "grid_i": Int32Col(shape=(), dflt=0, pos=3),
  "grid_j": Int32Col(shape=(), dflt=0, pos=4),
  "idnumber": Int64Col(shape=(), dflt=0, pos=5),
  "name": StringCol(itemsize=16, shape=(), dflt=b'', pos=6),
  "pressure": Float32Col(shape=(), dflt=0.0, pos=7)}
  byteorder := 'little'
  chunkshape := (1394,)

Zuweisung das particle einer Spalte in der Tabelle entspricht

In [35]:
particle = table.row
particle

/detector/readout.row (Row), pointing to row #0

Zuweisungen zu den Spalten<br>
append Speichert die Datei im Arbeitsspeicher an die Tabelle

In [36]:
for i in range(10):
    particle['name']  = 'Particle: %6d' % (i)
    particle['TDCcount'] = i % 256
    particle['ADCcount'] = (i * 256) % (1 << 16)
    particle['grid_i'] = i
    particle['grid_j'] = 10 - i
    particle['pressure'] = float(i*i)
    particle['energy'] = float(particle['pressure'] ** 4)
    particle['idnumber'] = i * (2 ** 34)
    # Insert a new particle record
    particle.append()

flush speichert die Datei auf der Festplatte und gibt den belegten Platz im Arbeitsspeicher frei

In [37]:
table.flush()

In [38]:
h5file

File(filename=tutorial1.h5, title='Test file', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) 'Test file'
/detector (Group) 'Detector information'
/detector/readout (Table(10,)) 'Readout example'
  description := {
  "ADCcount": UInt16Col(shape=(), dflt=0, pos=0),
  "TDCcount": UInt8Col(shape=(), dflt=0, pos=1),
  "energy": Float64Col(shape=(), dflt=0.0, pos=2),
  "grid_i": Int32Col(shape=(), dflt=0, pos=3),
  "grid_j": Int32Col(shape=(), dflt=0, pos=4),
  "idnumber": Int64Col(shape=(), dflt=0, pos=5),
  "name": StringCol(itemsize=16, shape=(), dflt=b'', pos=6),
  "pressure": Float32Col(shape=(), dflt=0.0, pos=7)}
  byteorder := 'little'
  chunkshape := (1394,)

alle Daten zu der Tabelle sind auch einzeln Aufrufbar

In [39]:
print('Spalten:')
print(table.colnames)
print('Datentypen:')
print(table.coltypes)
print('Anzahl Reihen:')
print(table.nrows)

Spalten:
['ADCcount', 'TDCcount', 'energy', 'grid_i', 'grid_j', 'idnumber', 'name', 'pressure']
Datentypen:
{'ADCcount': 'uint16', 'TDCcount': 'uint8', 'energy': 'float64', 'grid_i': 'int32', 'grid_j': 'int32', 'idnumber': 'int64', 'name': 'string', 'pressure': 'float32'}
Anzahl Reihen:
10


Filter nach x['TDCcount'] > 3 and 20 <= x['pressure'] < 50]

In [40]:
table = h5file.root.detector.readout
pressure = [x['pressure'] for x in table.iterrows() if x['TDCcount'] > 3 and 20 <= x['pressure'] < 50]
pressure

[25.0, 36.0, 49.0]

In [41]:
table = h5file.root.detector.readout
name = [x['name'] for x in table.iterrows() if x['TDCcount'] > 3 and 20 <= x['pressure'] < 50]
name

[b'Particle:      5', b'Particle:      6', b'Particle:      7']

Filter auf Bedingung

In [42]:
for record in table.where("pressure == 25.0"):
    print(record.fetch_all_fields())
    

(1280, 5,  390625., 5, 5, 85899345920, b'Particle:      5',  25.)


## Arrays

In [15]:
gcolumns = h5file.create_group(h5file.root, "columns", "Pressure and Name")
h5file

File(filename=tutorial1.h5, title='Test file', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) 'Test file'
/columns (Group) 'Pressure and Name'
/detector (Group) 'Detector information'
/detector/readout (Table(10,)) 'Readout example'
  description := {
  "ADCcount": UInt16Col(shape=(), dflt=0, pos=0),
  "TDCcount": UInt8Col(shape=(), dflt=0, pos=1),
  "energy": Float64Col(shape=(), dflt=0.0, pos=2),
  "grid_i": Int32Col(shape=(), dflt=0, pos=3),
  "grid_j": Int32Col(shape=(), dflt=0, pos=4),
  "idnumber": Int64Col(shape=(), dflt=0, pos=5),
  "name": StringCol(itemsize=16, shape=(), dflt=b'', pos=6),
  "pressure": Float32Col(shape=(), dflt=0.0, pos=7)}
  byteorder := 'little'
  chunkshape := (1394,)

In [16]:
h5file.create_array(gcolumns, 'pressure', pressure, "Pressure column selection")

/columns/pressure (Array(3,)) 'Pressure column selection'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'python'
  byteorder := 'little'
  chunkshape := None

In [17]:
h5file.create_array(gcolumns, 'name', name, "Name column selection")

/columns/name (Array(3,)) 'Name column selection'
  atom := StringAtom(itemsize=16, shape=(), dflt=b'')
  maindim := 0
  flavor := 'python'
  byteorder := 'irrelevant'
  chunkshape := None

In [18]:
h5file

File(filename=tutorial1.h5, title='Test file', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) 'Test file'
/columns (Group) 'Pressure and Name'
/columns/name (Array(3,)) 'Name column selection'
  atom := StringAtom(itemsize=16, shape=(), dflt=b'')
  maindim := 0
  flavor := 'python'
  byteorder := 'irrelevant'
  chunkshape := None
/columns/pressure (Array(3,)) 'Pressure column selection'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'python'
  byteorder := 'little'
  chunkshape := None
/detector (Group) 'Detector information'
/detector/readout (Table(10,)) 'Readout example'
  description := {
  "ADCcount": UInt16Col(shape=(), dflt=0, pos=0),
  "TDCcount": UInt8Col(shape=(), dflt=0, pos=1),
  "energy": Float64Col(shape=(), dflt=0.0, pos=2),
  "grid_i": Int32Col(shape=(), dflt=0, pos=3),
  "grid_j": Int32Col(shape=(), dflt=0, pos=4),
  "idnumber": Int64Col(shape=(), dflt=0, pos=5),
 

Walk_groups ermöglicht es alle Gruppen im Speicher zu besuchen und list_nodes alle Knoten in einem Speicher

In [19]:
for group in h5file.walk_groups("/"):
    for array in h5file.list_nodes(group, classname='Array'):
        print(array)

/columns/name (Array(3,)) 'Name column selection'
/columns/pressure (Array(3,)) 'Pressure column selection'


lese Werte der Arrays aus

In [21]:
pressureObject = h5file.get_node("/columns", "pressure")
pressureArray = pressureObject.read()
pressureArray

[25.0, 36.0, 49.0]

In [22]:
nameArray = h5file.root.columns.name.read()
nameArray

[b'Particle:      5', b'Particle:      6', b'Particle:      7']

Beispiel um die Arrays zu lesen

In [23]:
for i in range(pressureObject.shape[0]):
    print(nameArray[i], "-->", pressureArray[i])

b'Particle:      5' --> 25.0
b'Particle:      6' --> 36.0
b'Particle:      7' --> 49.0


## Modify your Data

In [24]:
for r in table.iterrows():
    print(r.fetch_all_fields())

(0, 0,  0., 0, 10, 0, b'Particle:      0',  0.)
(256, 1,  1., 1, 9, 17179869184, b'Particle:      1',  1.)
(512, 2,  256., 2, 8, 34359738368, b'Particle:      2',  4.)
(768, 3,  6561., 3, 7, 51539607552, b'Particle:      3',  9.)
(1024, 4,  65536., 4, 6, 68719476736, b'Particle:      4',  16.)
(1280, 5,  390625., 5, 5, 85899345920, b'Particle:      5',  25.)
(1536, 6,  1679616., 6, 4, 103079215104, b'Particle:      6',  36.)
(1792, 7,  5764801., 7, 3, 120259084288, b'Particle:      7',  49.)
(2048, 8,  16777216., 8, 2, 137438953472, b'Particle:      8',  64.)
(2304, 9,  43046721., 9, 1, 154618822656, b'Particle:      9',  81.)


Hinter energy wird die Spalte angegeben

In [25]:
table.cols.energy[2] = 2
for r in table.iterrows():
    print(r.fetch_all_fields())

(0, 0,  0., 0, 10, 0, b'Particle:      0',  0.)
(256, 1,  1., 1, 9, 17179869184, b'Particle:      1',  1.)
(512, 2,  2., 2, 8, 34359738368, b'Particle:      2',  4.)
(768, 3,  6561., 3, 7, 51539607552, b'Particle:      3',  9.)
(1024, 4,  65536., 4, 6, 68719476736, b'Particle:      4',  16.)
(1280, 5,  390625., 5, 5, 85899345920, b'Particle:      5',  25.)
(1536, 6,  1679616., 6, 4, 103079215104, b'Particle:      6',  36.)
(1792, 7,  5764801., 7, 3, 120259084288, b'Particle:      7',  49.)
(2048, 8,  16777216., 8, 2, 137438953472, b'Particle:      8',  64.)
(2304, 9,  43046721., 9, 1, 154618822656, b'Particle:      9',  81.)


step ist der Abstand zwischen den bearbeiteten Daten

In [26]:
table.modify_rows(start=1, step=3,rows=[(1, 2, 3.0, 4, 5, 6, 'Particle:   None', 8.0),(2, 4, 6.0, 8, 10, 12, 'Particle: None*2', 16.0)])
for r in table.iterrows():
    print(r.fetch_all_fields())

(0, 0,  0., 0, 10, 0, b'Particle:      0',  0.)
(1, 2,  3., 4, 5, 6, b'Particle:   None',  8.)
(512, 2,  2., 2, 8, 34359738368, b'Particle:      2',  4.)
(768, 3,  6561., 3, 7, 51539607552, b'Particle:      3',  9.)
(2, 4,  6., 8, 10, 12, b'Particle: None*2',  16.)
(1280, 5,  390625., 5, 5, 85899345920, b'Particle:      5',  25.)
(1536, 6,  1679616., 6, 4, 103079215104, b'Particle:      6',  36.)
(1792, 7,  5764801., 7, 3, 120259084288, b'Particle:      7',  49.)
(2048, 8,  16777216., 8, 2, 137438953472, b'Particle:      8',  64.)
(2304, 9,  43046721., 9, 1, 154618822656, b'Particle:      9',  81.)


In [27]:
h5file.close()

close schliesst die Datei und andere können darauf zugreifen. Solange sie noch offen ist kann kein anderer zugreifen.<br>
Mit dem Tool ViTables kann man sich die Informationen anschauen wie ein DataFrame

<img src='ViTable.png'>

## Pandas und HDF

Angeben von Dateiname und dem Pfad drunter<br>
Funktioniert nur mit Tables nicht mit Arrays

In [43]:
import pandas as pd
df = pd.read_hdf('tutorial1.h5', 'detector/readout')
df

,ADCcount,TDCcount,energy,grid_i,grid_j,idnumber,name,pressure
0,0,0,0.0,0,10,0,Particle: 0,0.0
1,256,1,1.0,1,9,17179869184,Particle: 1,1.0
2,512,2,256.0,2,8,34359738368,Particle: 2,4.0
3,768,3,6561.0,3,7,51539607552,Particle: 3,9.0
4,1024,4,65536.0,4,6,68719476736,Particle: 4,16.0
5,1280,5,390625.0,5,5,85899345920,Particle: 5,25.0
6,1536,6,1679616.0,6,4,103079215104,Particle: 6,36.0
7,1792,7,5764801.0,7,3,120259084288,Particle: 7,49.0
8,2048,8,16777216.0,8,2,137438953472,Particle: 8,64.0
9,2304,9,43046721.0,9,1,154618822656,Particle: 9,81.0
